In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
976783,2021-02-07,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
976784,2021-02-08,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
976785,2021-02-09,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
976786,2021-02-10,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
26635,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
26637,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
26639,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
26641,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
26643,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
606383,2021-02-07,Arkansas,Arkansas,5001,1881,37.00,5000,Arkansas,AR,Arkansas,State,3017804
606385,2021-02-08,Arkansas,Arkansas,5001,1884,37.00,5000,Arkansas,AR,Arkansas,State,3017804
606387,2021-02-09,Arkansas,Arkansas,5001,1894,37.00,5000,Arkansas,AR,Arkansas,State,3017804
606389,2021-02-10,Arkansas,Arkansas,5001,1908,37.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
976783,2021-02-07,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
976784,2021-02-08,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
976785,2021-02-09,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
976786,2021-02-10,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-02-11') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
387,2021-02-11,Snohomish,Washington,29578,512.00,53061,WA,County,822083,3597.93,62.28
772,2021-02-11,Cook,Illinois,463581,9655.00,17031,IL,County,5150233,9001.17,187.47
1156,2021-02-11,Orange,California,255254,3493.00,6059,CA,County,3175692,8037.74,109.99
1539,2021-02-11,Maricopa,Arizona,495621,8334.00,4013,AZ,County,4485414,11049.62,185.80
1922,2021-02-11,Los Angeles,California,1159135,18658.00,6037,CA,County,10039107,11546.20,185.85
...,...,...,...,...,...,...,...,...,...,...,...
976423,2021-02-11,Wheeler,Oregon,22,1.00,41069,OR,County,1332,1651.65,75.08
976545,2021-02-11,King,Texas,13,0.00,48269,TX,County,272,4779.41,0.00
976636,2021-02-11,Esmeralda,Nevada,37,0.00,32009,NV,County,873,4238.26,0.00
976723,2021-02-11,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
387,2021-02-11,Snohomish,Washington,29578,512.00,53061,WA,County,822083,3597.93,62.28,62.28,3597.93
772,2021-02-11,Cook,Illinois,463581,9655.00,17031,IL,County,5150233,9001.17,187.47,187.47,9001.17
1156,2021-02-11,Orange,California,255254,3493.00,6059,CA,County,3175692,8037.74,109.99,109.99,8037.74
1539,2021-02-11,Maricopa,Arizona,495621,8334.00,4013,AZ,County,4485414,11049.62,185.80,185.80,11049.62
1922,2021-02-11,Los Angeles,California,1159135,18658.00,6037,CA,County,10039107,11546.20,185.85,185.85,11546.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...
976423,2021-02-11,Wheeler,Oregon,22,1.00,41069,OR,County,1332,1651.65,75.08,75.08,1651.65
976545,2021-02-11,King,Texas,13,0.00,48269,TX,County,272,4779.41,0.00,0.00,4779.41
976636,2021-02-11,Esmeralda,Nevada,37,0.00,32009,NV,County,873,4238.26,0.00,0.00,4238.26
976723,2021-02-11,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00,0.00,591.72


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
689653,2021-02-11,Clarke,Alabama,3367,43.00,1025,AL,County,23622,14253.66,182.03,182.03,14253.66,1
633908,2021-02-11,Hale,Alabama,2011,57.00,1065,AL,County,14651,13726.03,389.05,389.05,13726.03,2
496581,2021-02-11,Lowndes,Alabama,1283,42.00,1085,AL,County,9726,13191.45,431.83,431.83,13191.45,3
440535,2021-02-11,Etowah,Alabama,12889,288.00,1055,AL,County,102268,12603.16,281.61,281.61,12603.16,4
369936,2021-02-11,Franklin,Alabama,3937,74.00,1059,AL,County,31362,12553.41,235.95,235.95,12553.41,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605110,2021-02-11,Converse,Wyoming,935,15.00,56009,WY,County,13822,6764.58,108.52,108.52,6764.58,19
929742,2021-02-11,Platte,Wyoming,563,11.00,56031,WY,County,8393,6707.97,131.06,131.06,6707.97,20
835331,2021-02-11,Niobrara,Wyoming,147,2.00,56027,WY,County,2356,6239.39,84.89,84.89,6239.39,21
751275,2021-02-11,Lincoln,Wyoming,1167,11.00,56023,WY,County,19830,5885.02,55.47,55.47,5885.02,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
496581,2021-02-11,Lowndes,Alabama,1283,42.00,1085,AL,County,9726,13191.45,431.83,431.83,13191.45,3,1
439239,2021-02-11,Clay,Alabama,1386,53.00,1027,AL,County,13235,10472.23,400.45,400.45,10472.23,24,2
535338,2021-02-11,Greene,Alabama,840,32.00,1063,AL,County,8111,10356.31,394.53,394.53,10356.31,25,3
633908,2021-02-11,Hale,Alabama,2011,57.00,1065,AL,County,14651,13726.03,389.05,389.05,13726.03,2,4
495935,2021-02-11,Crenshaw,Alabama,1384,51.00,1041,AL,County,13772,10049.38,370.32,370.32,10049.38,31,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
534049,2021-02-11,Hot Springs,Wyoming,357,3.00,56017,WY,County,4413,8089.73,67.98,67.98,8089.73,16,19
732690,2021-02-11,Uinta,Wyoming,1992,12.00,56041,WY,County,20226,9848.71,59.33,59.33,9848.71,6,20
751275,2021-02-11,Lincoln,Wyoming,1167,11.00,56023,WY,County,19830,5885.02,55.47,55.47,5885.02,22,21
210898,2021-02-11,Teton,Wyoming,3256,8.00,56039,WY,County,23464,13876.58,34.09,34.09,13876.58,1,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,62.28,3597.93,15,22
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,62.28,3597.93,15,22
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,62.28,3597.93,15,22
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,62.28,3597.93,15,22
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,62.28,3597.93,15,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
974473,2021-02-07,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
974474,2021-02-08,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
974475,2021-02-09,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
974476,2021-02-10,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
687027,2020-04-01,Clarke,Alabama,2,0.00,1025,AL,County,23622,8.47,0.00,182.03,14253.66,52,1,2.00
687028,2020-04-02,Clarke,Alabama,2,0.00,1025,AL,County,23622,8.47,0.00,182.03,14253.66,52,1,0.00
687029,2020-04-03,Clarke,Alabama,3,0.00,1025,AL,County,23622,12.70,0.00,182.03,14253.66,52,1,1.00
687030,2020-04-04,Clarke,Alabama,7,0.00,1025,AL,County,23622,29.63,0.00,182.03,14253.66,52,1,4.00
687031,2020-04-05,Clarke,Alabama,9,0.00,1025,AL,County,23622,38.10,0.00,182.03,14253.66,52,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
763719,2021-02-07,Crook,Wyoming,417,10.00,56011,WY,County,7584,5498.42,131.86,145.04,5498.42,6,23,0.00
763720,2021-02-08,Crook,Wyoming,417,10.00,56011,WY,County,7584,5498.42,131.86,145.04,5498.42,6,23,0.00
763721,2021-02-09,Crook,Wyoming,417,11.00,56011,WY,County,7584,5498.42,145.04,145.04,5498.42,6,23,0.00
763722,2021-02-10,Crook,Wyoming,417,11.00,56011,WY,County,7584,5498.42,145.04,145.04,5498.42,6,23,0.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
494591,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,431.83,13191.45,1,3,1.00,0.00
494592,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,431.83,13191.45,1,3,0.00,0.00
494593,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,431.83,13191.45,1,3,0.00,0.00
494594,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,431.83,13191.45,1,3,0.00,0.00
494595,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,431.83,13191.45,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492971,2021-02-07,Albany,Wyoming,3825,11.00,56001,WY,County,38880,9837.96,28.29,28.29,9889.40,23,5,2.00,0.00
492972,2021-02-08,Albany,Wyoming,3831,11.00,56001,WY,County,38880,9853.40,28.29,28.29,9889.40,23,5,6.00,0.00
492973,2021-02-09,Albany,Wyoming,3832,11.00,56001,WY,County,38880,9855.97,28.29,28.29,9889.40,23,5,1.00,0.00
492974,2021-02-10,Albany,Wyoming,3838,11.00,56001,WY,County,38880,9871.40,28.29,28.29,9889.40,23,5,6.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-02-11') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
252776,2021-02-11,Imperial,California,26589,583.00,6025,CA,County,181215,14672.63,321.72,321.72,14672.63,1,2,40.00,10.00
441783,2021-02-11,Inyo,California,1138,34.00,6027,CA,County,18039,6308.55,188.48,188.48,6308.55,2,32,1.00,0.00
1922,2021-02-11,Los Angeles,California,1159135,18658.00,6037,CA,County,10039107,11546.20,185.85,185.85,11546.20,3,5,3314.00,158.00
56569,2021-02-11,Stanislaus,California,53614,888.00,6099,CA,County,550660,9736.32,161.26,161.26,9736.32,4,10,226.00,7.00
70086,2021-02-11,Tulare,California,45782,672.00,6107,CA,County,466195,9820.35,144.15,144.15,9820.35,5,9,78.00,11.00
26292,2021-02-11,Riverside,California,284814,3477.00,6065,CA,County,2470546,11528.38,140.74,140.74,11528.38,6,6,646.00,23.00
400525,2021-02-11,Merced,California,27701,375.00,6047,CA,County,277680,9975.87,135.05,135.05,9975.87,7,8,139.00,2.00
535280,2021-02-11,Kings,California,21443,205.00,6031,CA,County,152940,14020.53,134.04,134.04,14020.53,8,3,83.00,5.00
25608,2021-02-11,Fresno,California,91876,1283.00,6019,CA,County,999101,9195.87,128.42,128.42,9195.87,9,14,146.00,25.00
45045,2021-02-11,San Joaquin,California,64709,965.00,6077,CA,County,762148,8490.35,126.62,126.62,8490.35,10,18,252.00,22.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
929293,2021-02-11,Lassen,California,5498,19.00,6035,CA,County,30573,17983.19,62.15,62.15,17983.19,34,1,0.00,0.00
252776,2021-02-11,Imperial,California,26589,583.00,6025,CA,County,181215,14672.63,321.72,321.72,14672.63,1,2,40.00,10.00
535280,2021-02-11,Kings,California,21443,205.00,6031,CA,County,152940,14020.53,134.04,134.04,14020.53,8,3,83.00,5.00
126174,2021-02-11,San Bernardino,California,280068,2243.00,6071,CA,County,2180085,12846.66,102.89,102.89,12846.66,13,4,395.00,59.00
1922,2021-02-11,Los Angeles,California,1159135,18658.00,6037,CA,County,10039107,11546.20,185.85,185.85,11546.20,3,5,3314.00,158.00
26292,2021-02-11,Riverside,California,284814,3477.00,6065,CA,County,2470546,11528.38,140.74,140.74,11528.38,6,6,646.00,23.00
158517,2021-02-11,Kern,California,99087,677.00,6029,CA,County,900202,11007.20,75.21,75.21,11007.20,26,7,451.00,13.00
400525,2021-02-11,Merced,California,27701,375.00,6047,CA,County,277680,9975.87,135.05,135.05,9975.87,7,8,139.00,2.00
70086,2021-02-11,Tulare,California,45782,672.00,6107,CA,County,466195,9820.35,144.15,144.15,9820.35,5,9,78.00,11.00
56569,2021-02-11,Stanislaus,California,53614,888.00,6099,CA,County,550660,9736.32,161.26,161.26,9736.32,4,10,226.00,7.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
252776,2021-02-11,Imperial,California,26589,583.00,6025,CA,County,181215,14672.63,321.72,321.72,14672.63,1,2,40.00,10.00
441783,2021-02-11,Inyo,California,1138,34.00,6027,CA,County,18039,6308.55,188.48,188.48,6308.55,2,32,1.00,0.00
1922,2021-02-11,Los Angeles,California,1159135,18658.00,6037,CA,County,10039107,11546.20,185.85,185.85,11546.20,3,5,3314.00,158.00
56569,2021-02-11,Stanislaus,California,53614,888.00,6099,CA,County,550660,9736.32,161.26,161.26,9736.32,4,10,226.00,7.00
70086,2021-02-11,Tulare,California,45782,672.00,6107,CA,County,466195,9820.35,144.15,144.15,9820.35,5,9,78.00,11.00
26292,2021-02-11,Riverside,California,284814,3477.00,6065,CA,County,2470546,11528.38,140.74,140.74,11528.38,6,6,646.00,23.00
400525,2021-02-11,Merced,California,27701,375.00,6047,CA,County,277680,9975.87,135.05,135.05,9975.87,7,8,139.00,2.00
535280,2021-02-11,Kings,California,21443,205.00,6031,CA,County,152940,14020.53,134.04,134.04,14020.53,8,3,83.00,5.00
25608,2021-02-11,Fresno,California,91876,1283.00,6019,CA,County,999101,9195.87,128.42,128.42,9195.87,9,14,146.00,25.00
45045,2021-02-11,San Joaquin,California,64709,965.00,6077,CA,County,762148,8490.35,126.62,126.62,8490.35,10,18,252.00,22.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,California,4,0.00,6025,CA,County,181215,2.21,0.00,321.72,14672.63,1,2,4.00,0.00
1,03/21/20,Imperial,California,4,0.00,6025,CA,County,181215,2.21,0.00,321.72,14672.63,1,2,0.00,0.00
2,03/22/20,Imperial,California,4,0.00,6025,CA,County,181215,2.21,0.00,321.72,14672.63,1,2,0.00,0.00
3,03/23/20,Imperial,California,4,0.00,6025,CA,County,181215,2.21,0.00,321.72,14672.63,1,2,0.00,0.00
4,03/24/20,Imperial,California,9,0.00,6025,CA,County,181215,4.97,0.00,321.72,14672.63,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3302,02/07/21,Lassen,California,5435,18.00,6035,CA,County,30573,17777.12,58.88,62.15,17983.19,34,1,0.00,0.00
3303,02/08/21,Lassen,California,5489,18.00,6035,CA,County,30573,17953.75,58.88,62.15,17983.19,34,1,54.00,0.00
3304,02/09/21,Lassen,California,5492,18.00,6035,CA,County,30573,17963.56,58.88,62.15,17983.19,34,1,3.00,0.00
3305,02/10/21,Lassen,California,5498,19.00,6035,CA,County,30573,17983.19,62.15,62.15,17983.19,34,1,6.00,1.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,California,4,0.00,6025,CA,County,181215,2.21,0.00,321.72,14672.63,1,2,4.00,0.00
1,03/21/20,Imperial,California,4,0.00,6025,CA,County,181215,2.21,0.00,321.72,14672.63,1,2,0.00,0.00
2,03/22/20,Imperial,California,4,0.00,6025,CA,County,181215,2.21,0.00,321.72,14672.63,1,2,0.00,0.00
3,03/23/20,Imperial,California,4,0.00,6025,CA,County,181215,2.21,0.00,321.72,14672.63,1,2,0.00,0.00
4,03/24/20,Imperial,California,9,0.00,6025,CA,County,181215,4.97,0.00,321.72,14672.63,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3376,02/07/21,San Joaquin,California,63619,893.00,6077,CA,County,762148,8347.33,117.17,126.62,8490.35,10,18,0.00,0.00
3377,02/08/21,San Joaquin,California,64138,923.00,6077,CA,County,762148,8415.43,121.11,126.62,8490.35,10,18,519.00,30.00
3378,02/09/21,San Joaquin,California,64396,929.00,6077,CA,County,762148,8449.28,121.89,126.62,8490.35,10,18,258.00,6.00
3379,02/10/21,San Joaquin,California,64457,943.00,6077,CA,County,762148,8457.28,123.73,126.62,8490.35,10,18,61.00,14.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)